# Research Question: 

Question: Can we predict the number of people who attain their bachelor's in an OECD country? 

In this assignment, we want to predict the number of people in OECD countrys that complete their Bachelor's degree. Facors like the country's GDP, ratio of people enrolled in primary education, secondary education, and tertiary education, the population of the country, how much the government spends on higher education, how much households spend on higher education, number of public universities, number of private universities, average cost of higher education, household income, and the year are all included as variables in this model. We will train a multivariate regression to see if we can reliably predict the number of people graduating with their bachelors.

Our inspiration for this project stemmed from https://icfdn.org/our-impact/education/. In this website, we found out that "Just one extra year of schooling can increase an individual’s earnings by up to 10%, and can raise the region’s average annual gross domestic product (GDP) growth by 0.37%." Because of this we wanted to explore how factors of education in a country can impact a country's GDP. 

We decided to only evaluate countries that are part of the Organization for Economic Co-operation and Development (OECD). There are 37 countries that are part of this organization that collaborate to develop policy standards and economic growth. We chose countries that are part of the OECD to evalvuate on because they account for three-fifths of the world's GDP, three-quarters of world trade, half of the world's energy consumption, and 18 percent of the world's population. Because these 37 countries account for a huge part of a country's GDP, we decided that this group of countries would be easier to evaluate compared attaining data from all 197 countries in the world.  https://www.state.gov/the-organization-for-economic-co-operation-and-development-oecd/#:~:text=and%20Development%20(OECD)-,The%20Organization%20for%20Economic%20Cooperation%20and%20Development%20(OECD),to%20promote%20sustainable%20economic%20growth.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb 

# Data Collection/Cleaning: 

## Enrollment rates in early childhood education: 
https://data.oecd.org/students/enrolment-rate-in-early-childhood-education.htm

This data set shows the enrollment rates in early childhood education in OECD countries. The net enrollment rates are calculated by the number of students of a particular age group (ages 3-5) enrolled in early childhood education by the size of the population of that age group. This data set only includes enrollment rates from 2013-2020.

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location would specificy what country. The indicator specified what type of education that age group was enrolled in. This column would be helpful for us because when we combine all the data sets, we need to use this indicator value to differentiate what type of education they are enrolled in. The subject column listed the age group of each row (ages 3-5). The Measure column just specifies how the data was separated. Because this data set is a part of a larger Education database, the measure value just indicated how each age group was grouped by, and in this case it is age. Because the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each enrollment rate is from, ranges from years 2013 to 2020. The Value column is displays the net enrollment percentages for each row. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, I removed the Measure, Frequency, and Flag Codes columns. After removing these columns, the data set only includes the columns: Location, Indicator, Subject, Measure, Frequency, and Time. 


In [4]:
enrollment_early_df = pd.read_csv('enrollment rate in early childhood education.csv')
query = """
        SELECT Location, Indicator, Subject, Time, Value
        FROM enrollment_early_df
        """
enrollment_early_df = duckdb.sql(query).df()
enrollment_early_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
0,AUS,ENROLMENT_ECE,AGE_3,2013,61.742
1,AUS,ENROLMENT_ECE,AGE_3,2014,68.133
2,AUS,ENROLMENT_ECE,AGE_3,2015,68.588
3,AUS,ENROLMENT_ECE,AGE_3,2016,63.574
4,AUS,ENROLMENT_ECE,AGE_3,2017,65.912


Because, we want to figure out the enrollment rates for each year for each country, we need to combine all the age groups for each year for each country. To do this, we calculated the average enrollment rate for each country for each year. At the end, we dropped the Subject and Indicator columns because for this specific case we just need the enrollment rates, years, and country name. We also renamed all the columns to better represent the values in order to better represent the data and easier to understand. 

In [5]:
filtered_df = enrollment_early_df[enrollment_early_df['SUBJECT'].isin(['AGE_3', 'AGE_4', 'AGE_5'])]
enrollment_early_education_df = filtered_df.groupby(['LOCATION','TIME'])['Value'].mean().reset_index()
enrollment_early_education_df = enrollment_early_education_df.drop_duplicates()
enrollment_early_education_df = enrollment_early_education_df.rename(columns = {"LOCATION": "Country", 
                                                                                "TIME": "Year", 
                                                                               "Value" : "Early Childhood Education Enrollment Rates"})
enrollment_early_education_df = enrollment_early_education_df.replace({"ENROLMENT_ECE" : "Early Childhood"})
enrollment_early_education_df.head()

,Country,Year,Early Childhood Education Enrollment Rates
0,AUS,2013,61.742
1,AUS,2014,68.133
2,AUS,2015,68.588
3,AUS,2016,63.574
4,AUS,2017,65.912


## Enrollment rates in secondary and teritiary education: 

https://data.oecd.org/students/enrolment-rate-in-secondary-and-tertiary-education.htm#indicator-chart

This data set shows the enrollment rates in secondary and tertiary education in OECD countries. The net enrollment rates are calculated by dividing the number of students of a particular age enrolled in these levels of education by the size of the population at that age (ages 17-19). The data set only includes data from 2013-2020.

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specified what type of education that age group was enrolled in. This column would be helpful for us because when we combine all the data sets, we need to use this indicator value to differentiate what type of education they are enrolled in. The subject column listed the age group of each row (ages 17-19). The Measure column just specifies how the data was separated. Because this data set is a part of a larger Education database, the measure value just indicated how each age group was grouped by, and in this case it is age. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each enrollment rate is from, ranges from years 2013 to 2020. The Value column is displays the net enrollment percentages for each row. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, I removed the Measure, Frequency, and Flag Codes columns. After removing these columns, the data set only includes the columns: Location, Indicator, Subject, Measure, Frequency, and Time.

In [6]:
enrollment_higher_df = pd.read_csv('enrollment rates in secondary and tertiary education.csv')

query = """
        SELECT Location, Indicator, Subject, Time, Value
        FROM enrollment_higher_df
        """
enrollment_higher_df = duckdb.sql(query).df()
enrollment_higher_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
0,AUS,ENROLMENT,AGE_17,2013,89.698
1,AUS,ENROLMENT,AGE_17,2014,91.439
2,AUS,ENROLMENT,AGE_17,2015,97.712
3,AUS,ENROLMENT,AGE_17,2016,97.517
4,AUS,ENROLMENT,AGE_17,2017,96.935


Because, we want to figure out the enrollment rates for each year for each country, we need to combine all the age groups for each year for each country. To do this, we calculated the average enrollment rate for each country for each year. At the end, we dropped the Subject and Indicator columns, because in this case, we just need the enrollment rates, year, and country. We also renamed all the columns to better represent the values in order to better represent the data and easier to understand. 

In [7]:
filtered_df = enrollment_higher_df[enrollment_higher_df['SUBJECT'].isin(['AGE_17', 'AGE_18', 'AGE_19'])]
enrollment_higher_education_df = filtered_df.groupby(['LOCATION','TIME'])['Value'].mean().reset_index()
enrollment_higher_education_df = enrollment_higher_education_df.drop_duplicates()
enrollment_higher_education_df = enrollment_higher_education_df.rename(columns = {"LOCATION": "Country", 
                                                                                "TIME": "Year", 
                                                                               "Value" : "Higher Education Enrollment Rates"})
enrollment_higher_education_df = enrollment_higher_education_df.replace({"ENROLMENT" : "Higher"})
enrollment_higher_education_df.head()

,Country,Year,Higher Education Enrollment Rates
0,AUS,2013,89.698
1,AUS,2014,91.439
2,AUS,2015,97.712
3,AUS,2016,97.517
4,AUS,2017,96.935


The two dataframes about enrollment rates in education are combined below, so that we will be able to have a cohesive dataframe that shows enrollment rates in early, and higher education in 37 different countries, from 2013 to 2017. 

In [8]:
query = """
        SELECT *
        FROM enrollment_early_education_df
        FULL JOIN enrollment_higher_education_df
        ON enrollment_early_education_df.Country = enrollment_higher_education_df.Country
        AND enrollment_early_education_df.Year = enrollment_higher_education_df.Year;
        """
enrollment_rates_df = duckdb.sql(query).df()
enrollment_rates_df

,Country,Year,Early Childhood Education Enrollment Rates,Country_2,Year_2,Higher Education Enrollment Rates
0,AUS,2013.0,61.742,AUS,2013.0,89.698
1,AUS,2014.0,68.133,AUS,2014.0,91.439
2,AUS,2015.0,68.588,AUS,2015.0,97.712
3,AUS,2016.0,63.574,AUS,2016.0,97.517
4,AUS,2017.0,65.912,AUS,2017.0,96.935
...,...,...,...,...,...,...
297,NaN,NaN,NaN,USA,2015.0,90.508
298,NaN,NaN,NaN,IRL,2015.0,99.089
299,NaN,NaN,NaN,PRT,2013.0,94.369
300,NaN,NaN,NaN,USA,2014.0,84.238


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers.

In [9]:
# Fill missing values in Country with values from Country_2
enrollment_rates_df['Country'] = enrollment_rates_df['Country'].fillna(enrollment_rates_df['Country_2'])

# Fill missing values in Country_2 with values from Country
enrollment_rates_df['Country_2'] = enrollment_rates_df['Country_2'].fillna(enrollment_rates_df['Country'])

# Fill missing values in Year with values from Year_2
enrollment_rates_df['Year'] = enrollment_rates_df['Year'].fillna(enrollment_rates_df['Year_2'])

# Fill missing values in Year_2 with values from Year
enrollment_rates_df['Year_2'] = enrollment_rates_df['Year_2'].fillna(enrollment_rates_df['Year'])

#dropping Country_2 and Year_2 columns
enrollment_rates_df = enrollment_rates_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
enrollment_rates_df['Year'] = enrollment_rates_df['Year'].astype(int)

enrollment_rates_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates
0,AUS,2013,61.742,89.698
1,AUS,2014,68.133,91.439
2,AUS,2015,68.588,97.712
3,AUS,2016,63.574,97.517
4,AUS,2017,65.912,96.935
...,...,...,...,...
297,USA,2015,NaN,90.508
298,IRL,2015,NaN,99.089
299,PRT,2013,NaN,94.369
300,USA,2014,NaN,84.238


In [10]:
enrollment_rates_df.to_csv('enrollment_rates.csv', index=False)

## GDP:

https://data.oecd.org/gdp/gross-domestic-product-gdp.htm

This data set shows the nominal Gross Domestic Product(GDP) per capita of OEPD countries in US dollars from 1960 to 2022. The Gross domestic product is the standard measure of the value added created through the production of goods and servives in a country during a certain period. While the GDP per capita can be found by diving the total GDP by its population. It also measure the inclome earned from that population, the total amount spend on a final goods and services. 

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specifies what is being measured. This column is also not necessary because there is only one value being measured in this specific data set, it is redundant. The Measure column just specifies how the data was measured, in this case it was measured in US dollars. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each gdp value is from from, ranges from years 1960 to 2022. The Value column displays the nominal gdp value. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "GDP in US Dollars" in order to specify the value of it, which is needed when we combine all the data sets. 

We also need to limit the years to be in between the years 2013 to 2020 in order to be consistent with the previous data sets found and limit the amount of nan in the data. 

In [11]:
gdp_df = pd.read_csv('gdp.csv')
query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "GDP per Capita"
        FROM gdp_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
gdp_df = duckdb.sql(query).df()
gdp_df

,Country,Year,GDP per Capita
0,AUS,2013,47763.215900
1,AUS,2014,47606.754929
2,AUS,2015,47226.759745
3,AUS,2016,50136.798381
4,AUS,2017,50706.489861
...,...,...,...
299,CRI,2016,19119.336147
300,CRI,2017,20368.230268
301,CRI,2018,21312.713380
302,CRI,2019,22739.241909


In [12]:
gdp_df.to_csv('clean_gdp.csv', index=False)

The two dataframes (enrollment_rates_df and gdp_df) are combined below using a Full Join, so that we will have a cohesive data frame showing enrollment rates and gdp of each country from 2013-2020. 

In [13]:
query = """
        SELECT *
        FROM enrollment_rates_df
        FULL JOIN gdp_df
        ON enrollment_rates_df.Country = gdp_df.Country
        AND enrollment_rates_df.Year = gdp_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,Country_2,Year_2,GDP per Capita
0,AUS,2013.0,61.742,89.698,AUS,2013,47763.215900
1,AUS,2014.0,68.133,91.439,AUS,2014,47606.754929
2,AUS,2015.0,68.588,97.712,AUS,2015,47226.759745
3,AUS,2016.0,63.574,97.517,AUS,2016,50136.798381
4,AUS,2017.0,65.912,96.935,AUS,2017,50706.489861
...,...,...,...,...,...,...,...
299,BEL,2015.0,NaN,96.959,BEL,2015,46201.685891
300,PRT,2014.0,NaN,96.919,PRT,2014,28742.313125
301,EST,2015.0,86.862,94.468,EST,2015,29222.748082
302,NaN,NaN,NaN,NaN,COL,2013,13266.002148


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant, change all the float values in Year to be integers, and round the GDP values to two decimals to represent US dollars. 

In [14]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

#rounding the GDP values to 2 decimals
combined_df['GDP per Capita'] = combined_df['GDP per Capita'].round(2)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita
0,AUS,2013,61.742,89.698,47763.22
1,AUS,2014,68.133,91.439,47606.75
2,AUS,2015,68.588,97.712,47226.76
3,AUS,2016,63.574,97.517,50136.80
4,AUS,2017,65.912,96.935,50706.49
...,...,...,...,...,...
299,BEL,2015,NaN,96.959,46201.69
300,PRT,2014,NaN,96.919,28742.31
301,EST,2015,86.862,94.468,29222.75
302,COL,2013,NaN,NaN,13266.00


## Population:

https://data.oecd.org/pop/population.htm

This data set shows the total population of OECD country in millions of people from 1950 to 2022. The total population includes the following: national armed forces stationed abroad; merchant seamen at sea; diplomatic personnel located abroad; civilian aliens resident in the country; displaced persons resident in the country. However, it excludes the following: foreign armed forces stationed in the country; foreign diplomatic personnel located in the country; civilian aliens temporarily in the country.

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specifies what is being measured (population). This column is also not necessary because there is only one value being measured in this specific data set, it is redundant. The Measure column just specifies how the data was measured, in this case it is the population per millions of people. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each gdp value is from from, ranges from years 1950 to 2022. The Value column displays the total population per millions of people. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Population" in order to specify the value of it, which is needed when we combine all the data sets. 

We also need to limit Years to be in between 2013 to 2020 in order to be consistent with the previous data sets and limit the amount of missing values. 

In [15]:
population_df = pd.read_csv('population.csv')
population_df

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value * 1000000 AS "Total Population"
        FROM population_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
population_df = duckdb.sql(query).df()
population_df

,Country,Year,Total Population
0,AUS,2013,23128129.0
1,AUS,2014,23475686.0
2,AUS,2015,23815995.0
3,AUS,2016,24190907.0
4,AUS,2017,24594202.0
...,...,...,...
307,LTU,2016,2868231.0
308,LTU,2017,2828403.0
309,LTU,2018,2801543.0
310,LTU,2019,2794137.0


In [16]:
population_df.to_csv('clean_population.csv', index=False)

The two dataframes (combined_df and population_df) are combined below using a Full Join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population of each country from 2013-2020.

In [17]:
query = """
        SELECT *
        FROM combined_df
        FULL JOIN population_df
        ON combined_df.Country = population_df.Country
        AND combined_df.Year = population_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Country_2,Year_2,Total Population
0,AUS,2013.0,61.742,89.698,47763.22,AUS,2013,2.312813e+07
1,AUS,2014.0,68.133,91.439,47606.75,AUS,2014,2.347569e+07
2,AUS,2015.0,68.588,97.712,47226.76,AUS,2015,2.381600e+07
3,AUS,2016.0,63.574,97.517,50136.80,AUS,2016,2.419091e+07
4,AUS,2017.0,65.912,96.935,50706.49,AUS,2017,2.459420e+07
...,...,...,...,...,...,...,...,...
307,NaN,NaN,NaN,NaN,NaN,OECD,2019,1.363023e+09
308,NaN,NaN,NaN,NaN,NaN,OECD,2017,1.346876e+09
309,NaN,NaN,NaN,NaN,NaN,OECD,2015,1.330543e+09
310,NaN,NaN,NaN,NaN,NaN,OECD,2014,1.322342e+09


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers, 

In [18]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07
...,...,...,...,...,...,...
307,OECD,2019,NaN,NaN,NaN,1.363023e+09
308,OECD,2017,NaN,NaN,NaN,1.346876e+09
309,OECD,2015,NaN,NaN,NaN,1.330543e+09
310,OECD,2014,NaN,NaN,NaN,1.322342e+09


## Public Spending On Education:

https://data.oecd.org/eduresource/public-spending-on-education.htm#indicator-chart

This data shows the public spending on education as a percentage fo GDP for tertiary levels of education. Public spending on education includes direct expendiute on educational institutions, as well as education-related public susidies given to households and administered by educational institutions. Public entities include ministries other than ministries of education, local and regional governments, and other public agencies. Public spending includes expenditure on schools, universities and other public and private institutions delivering or supporting educational services. 

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specifies what is being measured (public spending on education). This column is also not necessary because there is only one value being measured in this specific data set, it is redundant. The Measure column just specifies how the data was measured, in this case it is the public spending as a percentage of gdp. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each gdp value is from from, ranges from years 2000 to 2020. The Value column displays the percentage of gdp that is used for public education. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Public Spending on Education" in order to specify the value of it, which is needed when we combine all the data sets.

We also need to limit Years to be in between 2013 to 2020 in order to be consistent with the previous data sets and limit the amount of missing values.

In [19]:
public_spending_df = pd.read_csv('public_spending_on_education.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Public Spending on Education (%)"
        FROM public_spending_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
public_spending_df = duckdb.sql(query).df()
public_spending_df

,Country,Year,Public Spending on Education (%)
0,AUS,2013,0.718812
1,AUS,2014,0.717300
2,AUS,2015,0.769290
3,AUS,2016,0.756591
4,AUS,2017,0.729282
...,...,...,...
300,LTU,2016,0.729869
301,LTU,2017,0.666880
302,LTU,2018,0.712819
303,LTU,2019,0.723762


In [20]:
public_spending_df.to_csv('clean_public_education_df.csv', index=False)

The two dataframes (combined_df and public_spending_df) are combined below using a Full Join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population, and public spending as a percentage of gdp of each country from 2013-2020.

In [21]:
query = """
        SELECT *
        FROM combined_df
        FULL JOIN public_spending_df
        ON combined_df.Country = public_spending_df.Country
        AND combined_df.Year = public_spending_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Country_2,Year_2,Public Spending on Education (%)
0,AUS,2013.0,61.742,89.698,47763.22,23128129.0,AUS,2013.0,0.718812
1,AUS,2014.0,68.133,91.439,47606.75,23475686.0,AUS,2014.0,0.717300
2,AUS,2015.0,68.588,97.712,47226.76,23815995.0,AUS,2015.0,0.769290
3,AUS,2016.0,63.574,97.517,50136.80,24190907.0,AUS,2016.0,0.756591
4,AUS,2017.0,65.912,96.935,50706.49,24594202.0,AUS,2017.0,0.729282
...,...,...,...,...,...,...,...,...,...
313,NaN,NaN,NaN,NaN,NaN,NaN,RUS,2013.0,0.824124
314,NaN,NaN,NaN,NaN,NaN,NaN,RUS,2017.0,0.547857
315,NaN,NaN,NaN,NaN,NaN,NaN,RUS,2015.0,0.734968
316,NaN,NaN,NaN,NaN,NaN,NaN,RUS,2014.0,0.811595


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values, or etc. and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers,

In [22]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%)
0,AUS,2013,61.742,89.698,47763.22,23128129.0,0.718812
1,AUS,2014,68.133,91.439,47606.75,23475686.0,0.717300
2,AUS,2015,68.588,97.712,47226.76,23815995.0,0.769290
3,AUS,2016,63.574,97.517,50136.80,24190907.0,0.756591
4,AUS,2017,65.912,96.935,50706.49,24594202.0,0.729282
...,...,...,...,...,...,...,...
313,RUS,2013,NaN,NaN,NaN,NaN,0.824124
314,RUS,2017,NaN,NaN,NaN,NaN,0.547857
315,RUS,2015,NaN,NaN,NaN,NaN,0.734968
316,RUS,2014,NaN,NaN,NaN,NaN,0.811595


## Private Spending on Education:

https://data.oecd.org/eduresource/private-spending-on-education.htm#indicator-chart

This data shows private spending on education as a percentage of GDP for tertiary education. Private spending on education refers to expenditure funded by private resources which are households and other private entities. It includes all direct expenditure on education institions, and net of public subsidies.

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specifies what is being measured (private spending on education). This column is also not necessary because there is only one value being measured in this specific data set, it is redundant. The Measure column just specifies how the data was measured, in this case it is the public spending as a percentage of gdp. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each gdp value is from from, ranges from years 2000 to 2020. The Value column displays the percentage of gdp that is used for public education. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Private Spending on Education (%)" in order to specify the value of it, which is needed when we combine all the data sets.

We also need to limit Years to be in between 2013 to 2020 in order to be consistent with the previous data sets and limit the amount of missing values.

In [23]:
private_spending_df = pd.read_csv('private_spending_on_education.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Private Spending on Education (%)"
        FROM private_spending_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
private_spending_df = duckdb.sql(query).df()
private_spending_df

,Country,Year,Private Spending on Education (%)
0,AUS,2013,0.972901
1,AUS,2014,1.133321
2,AUS,2015,1.264779
3,AUS,2016,1.157713
4,AUS,2017,1.287303
...,...,...,...
286,LTU,2016,0.338386
287,LTU,2017,0.310164
288,LTU,2018,0.304289
289,LTU,2019,0.298019


In [24]:
private_spending_df.to_csv('clean_private_education_df.csv', index=False)

The two dataframes (combined_df and private_spending_df) are combined below using a Full Join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population, and public spending as a percentage of GDP, and private spending as a percentage of GDP of each country from 2013-2020.

In [25]:
query = """
        SELECT *
        FROM combined_df
        FULL JOIN private_spending_df
        ON combined_df.Country = private_spending_df.Country
        AND combined_df.Year = private_spending_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Country_2,Year_2,Private Spending on Education (%)
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,AUS,2013.0,0.972901
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,AUS,2014.0,1.133321
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,AUS,2015.0,1.264779
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,AUS,2016.0,1.157713
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,AUS,2017.0,1.287303
...,...,...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN,NaN,NaN


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values, or etc. and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers,

In [26]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%)
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303
...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN


## Household Income per Capita:

https://data.oecd.org/hha/household-disposable-income.htm#indicator-chart

This data set shows the gross household disposable income per capita in 37 OECD countries. Household disposable income is available to households such as wages and salaries, income from self-employment and unincorporated enterprices, income from pensions and other social benefits, and income from financial investments. Gross means that depreciation costs are not subtracted. For gross household disposable income per capita, growth rates (percentage change from previous period) are presented; these are ‘real’ growth rates adjusted to remove the effects of price changes. Information is also presented for gross household disposable income including social transfers, such as health or education provided for free or at reduced prices by governments and not-for-profit organisations. 

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specifies what is being measured (household disposable income). This column is also not necessary because there is only one value being measured in this specific data set, it is redundant. The Measure column just specifies how the data was measured, in this case it is the household income per capita. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each gdp value is from from, ranges from years 1970 to 2020. The Value column displays the percentage of gdp that is used for public education. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Household Income per Capita" in order to specify the value of it, which is needed when we combine all the data sets.

We also need to limit Years to be in between 2013 to 2020 in order to be consistent with the previous data sets and limit the amount of missing values.

In [27]:
household_income_df = pd.read_csv('household_income.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Household Income per Capita"
        FROM household_income_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
household_income_df = duckdb.sql(query).df()
household_income_df

,Country,Year,Household Income per Capita
0,JPN,2013,30474.352272
1,JPN,2014,30032.209494
2,JPN,2015,30597.027634
3,JPN,2016,29290.345599
4,JPN,2017,29915.592624
...,...,...,...
271,CRI,2016,14675.888631
272,CRI,2017,16130.493739
273,CRI,2018,16619.155338
274,CRI,2019,17161.123623


In [28]:
household_income_df.to_csv('clean_household_income_df.csv', index=False)

The two dataframes (combined_df and household_income_df) are combined below using a Full Join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population, public spending as a percentage of GDP, private spending as a percentage of GDP, and household income per capita of each country from 2013-2020.

In [29]:
query = """
        SELECT *
        FROM combined_df
        FULL JOIN household_income_df
        ON combined_df.Country = household_income_df.Country
        AND combined_df.Year = household_income_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Country_2,Year_2,Household Income per Capita
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901,AUS,2013.0,36048.427633
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321,AUS,2014.0,36816.129278
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779,AUS,2015.0,37553.196701
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713,AUS,2016.0,38951.606333
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303,AUS,2017.0,38813.739701
...,...,...,...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN,NaN,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN,NaN,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN,NaN,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN,NaN,NaN,NaN


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values, or etc. and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers.

In [30]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901,36048.427633
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321,36816.129278
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779,37553.196701
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713,38951.606333
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303,38813.739701
...,...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN,NaN


## Education Spending:

https://data.oecd.org/eduresource/education-spending.htm#indicator-chart

This data set shows the average amount of education spending that covers expenditure on schools, universities and other public and private educational institutions in 37 OECD countries. Spending includes instruction and ancillary services for students and families provided through educational institutions. Education spending is shown in USD per student.

The original data set has 8 different columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specifies the country. The indicator specifies what is being measured (education spending in dollars). This column is also not necessary because there is only one value being measured in this specific data set, it is redundant. The Measure column just specifies how the data was measured, in this case it is the USD per student. Since the Measure is the same for all the rows, this column isn't necessary. The frequency column is similar to the Measure column and is also not necessary for this case. The Time column indicates what year each gdp value is from from, ranges from years 1995 to 2020. The Value column displays the percentage of gdp that is used for public education. And the last column, Flag Codes, is used for indicating something wrong for each row, in this case, because the whole column does not contain any values, this column is not necessary.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Average Spending on Higher Education (USD/student)" in order to specify the value of it, which is needed when we combine all the data sets.

We also need to limit Years to be in between 2013 to 2020 in order to be consistent with the previous data sets and limit the amount of missing values.

In [31]:
average_spending_df = pd.read_csv('educaion_spending.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Average Spending on Higher Education (USD/student)"
        FROM average_spending_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
average_spending_df = duckdb.sql(query).df()
average_spending_df

,Country,Year,Average Spending on Higher Education (USD/student)
0,AUT,2013,16853.000
1,AUT,2014,16867.620
2,AUT,2015,17560.620
3,AUT,2016,18625.000
4,AUT,2017,18974.750
...,...,...,...
285,LTU,2016,7852.196
286,LTU,2017,8412.116
287,LTU,2018,9908.427
288,LTU,2019,11431.870


In [32]:
average_spending_df.to_csv('clean_average_spending_df.csv', index=False)

The two dataframes (combined_df and average_spending_df) are combined below using a Full Join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population, public spending as a percentage of GDP, private spending as a percentage of GDP, household income per capita, and average spending for higher education per student of each country from 2013-2020.

In [33]:
query = """
        SELECT *
        FROM combined_df
        FULL JOIN average_spending_df
        ON combined_df.Country = average_spending_df.Country
        AND combined_df.Year = average_spending_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita,Country_2,Year_2,Average Spending on Higher Education (USD/student)
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901,36048.427633,AUS,2013.0,18252.76
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321,36816.129278,AUS,2014.0,19493.58
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779,37553.196701,AUS,2015.0,20304.37
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713,38951.606333,AUS,2016.0,16181.19
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303,38813.739701,AUS,2017.0,20273.53
...,...,...,...,...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN,NaN,NaN,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN,NaN,NaN,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN,NaN,NaN,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN,NaN,NaN,NaN,NaN


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values, or etc. and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers.

In [34]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita,Average Spending on Higher Education (USD/student)
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901,36048.427633,18252.76
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321,36816.129278,19493.58
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779,37553.196701,20304.37
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713,38951.606333,16181.19
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303,38813.739701,20273.53
...,...,...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN,NaN,NaN


## Number of Universities:

We weren't able to find a data set that directly listed the amount of universities in each OECD country. Because of this, we had to make a data set on Excel and convert to a csv later.

All the information from the data set was found from this website: https://www.webometrics.info/en/distribution_by_country


In [35]:
num_universities_df = pd.read_csv('number_of_universities.csv')
num_universities_df

,Country,Number of Universities
0,AUS,187
1,AUT,84
2,BEL,142
3,CAN,383
4,CHL,130
5,COL,299
6,CRI,68
7,CZE,64
8,DNK,81
9,EST,31


The two dataframes (combined_df and num_universities_df) are combined below using a Left Join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population, public spending as a percentage of GDP, private spending as a percentage of GDP, household income per capita, average spending for higher education per student, and number of universities for each country from 2013-2020.

In [36]:
combined_df = pd.merge(combined_df, num_universities_df, on='Country', how='left')

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita,Average Spending on Higher Education (USD/student),Number of Universities
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901,36048.427633,18252.76,187.0
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321,36816.129278,19493.58,187.0
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779,37553.196701,20304.37,187.0
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713,38951.606333,16181.19,187.0
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303,38813.739701,20273.53,187.0
...,...,...,...,...,...,...,...,...,...,...,...
313,OECD,2019,NaN,NaN,NaN,1.363023e+09,NaN,NaN,NaN,NaN,NaN
314,OECD,2017,NaN,NaN,NaN,1.346876e+09,NaN,NaN,NaN,NaN,NaN
315,OECD,2015,NaN,NaN,NaN,1.330543e+09,NaN,NaN,NaN,NaN,NaN
316,OECD,2014,NaN,NaN,NaN,1.322342e+09,NaN,NaN,NaN,NaN,NaN


Since some of the rows above in the combined data frame, have values for OECD, which represents the averages across all the countries, we are going to drop those rows, since not all the data frames that we joined in this value.

In [37]:
combined_df = combined_df[combined_df['Country'] != 'OECD']
combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita,Average Spending on Higher Education (USD/student),Number of Universities
0,AUS,2013,61.742,89.698,47763.22,2.312813e+07,0.718812,0.972901,36048.427633,18252.76,187.0
1,AUS,2014,68.133,91.439,47606.75,2.347569e+07,0.717300,1.133321,36816.129278,19493.58,187.0
2,AUS,2015,68.588,97.712,47226.76,2.381600e+07,0.769290,1.264779,37553.196701,20304.37,187.0
3,AUS,2016,63.574,97.517,50136.80,2.419091e+07,0.756591,1.157713,38951.606333,16181.19,187.0
4,AUS,2017,65.912,96.935,50706.49,2.459420e+07,0.729282,1.287303,38813.739701,20273.53,187.0
...,...,...,...,...,...,...,...,...,...,...,...
305,DNK,2015,96.956,90.792,49058.14,5.678348e+06,NaN,NaN,30507.182928,NaN,81.0
306,KOR,2013,92.717,94.807,34244.24,5.042889e+07,NaN,NaN,21573.319908,NaN,401.0
307,KOR,2014,90.008,94.532,35324.26,5.074666e+07,NaN,NaN,21915.955777,NaN,401.0
308,CRI,2017,5.414,NaN,20368.23,4.947490e+06,NaN,NaN,16130.493739,NaN,68.0


In [38]:
combined_df.to_csv('combined_data.csv', index=False)

In [39]:
students_per_teacher_df = pd.read_csv('students_per_teaching_staff.csv')

In [40]:
students_per_teacher_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUT,STUDPERTEACHER,EARLYCHILDEDU,RT,A,2013,12.798,NaN
1,AUT,STUDPERTEACHER,EARLYCHILDEDU,RT,A,2014,12.926,NaN
2,AUT,STUDPERTEACHER,EARLYCHILDEDU,RT,A,2015,12.537,NaN
3,AUT,STUDPERTEACHER,EARLYCHILDEDU,RT,A,2016,12.346,NaN
4,AUT,STUDPERTEACHER,EARLYCHILDEDU,RT,A,2017,12.709,NaN


In [41]:
query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Students per Teaching Staff"
        FROM students_per_teacher_df
        WHERE TIME IN (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020);
        """
students_per_teacher_df = duckdb.sql(query).df()
students_per_teacher_df

,Country,Year,Students per Teaching Staff
0,AUT,2013,12.798
1,AUT,2014,12.926
2,AUT,2015,12.537
3,AUT,2016,12.346
4,AUT,2017,12.709
...,...,...,...
200,SVN,2016,8.045
201,SVN,2017,16.580
202,SVN,2018,7.846
203,SVN,2019,16.625


In [42]:
students_per_teacher_df.to_csv('clean_students_per_teacher_df.csv', index=False)

In [43]:
query = """
        SELECT *
        FROM combined_df
        FULL JOIN students_per_teacher_df
        ON combined_df.Country = students_per_teacher_df.Country
        AND combined_df.Year = students_per_teacher_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita,Average Spending on Higher Education (USD/student),Number of Universities,Country_2,Year_2,Students per Teaching Staff
0,AUT,2014,72.884,89.160,48813.53,8543932.0,1.614541,0.095220,34326.229141,16867.620,84.0,AUT,2014.0,12.926
1,AUT,2016,75.783,86.831,52665.09,8739806.0,1.631474,0.112024,36227.955632,18625.000,84.0,AUT,2016.0,12.346
2,AUT,2017,76.462,86.632,54188.36,8795073.0,1.560725,0.152499,36984.312014,18974.750,84.0,AUT,2017.0,12.709
3,AUT,2018,77.280,87.348,56956.11,8837707.0,1.552325,0.182909,38190.870562,20416.620,84.0,AUT,2018.0,12.541
4,AUT,2019,77.815,87.949,59716.25,8877637.0,1.557053,0.192745,40065.855276,21946.230,84.0,AUT,2019.0,11.999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,SVK,2013,62.582,89.959,28021.09,5413392.5,0.797937,0.258909,19313.861573,10224.650,36.0,SVK,2013.0,12.634
306,SVN,2016,83.836,96.222,33942.77,2064241.0,0.843163,0.144241,23387.750562,11487.920,55.0,SVN,2016.0,8.045
307,JPN,2017,82.985,NaN,41531.22,126918546.0,0.425807,0.937593,29915.592624,18888.400,992.0,JPN,2017.0,14.515
308,HUN,2015,81.185,89.694,26798.85,9843025.0,0.545663,0.321258,17974.185766,8815.057,69.0,HUN,2015.0,12.438


In [44]:
# Fill missing values in Country with values from Country_2
combined_df['Country'] = combined_df['Country'].fillna(combined_df['Country_2'])

# Fill missing values in Country_2 with values from Country
combined_df['Country_2'] = combined_df['Country_2'].fillna(combined_df['Country'])

# Fill missing values in Year with values from Year_2
combined_df['Year'] = combined_df['Year'].fillna(combined_df['Year_2'])

# Fill missing values in Year_2 with values from Year
combined_df['Year_2'] = combined_df['Year_2'].fillna(combined_df['Year'])

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,Early Childhood Education Enrollment Rates,Higher Education Enrollment Rates,GDP per Capita,Total Population,Public Spending on Education (%),Private Spending on Education (%),Household Income per Capita,Average Spending on Higher Education (USD/student),Number of Universities,Students per Teaching Staff
0,AUT,2014,72.884,89.160,48813.53,8543932.0,1.614541,0.095220,34326.229141,16867.620,84.0,12.926
1,AUT,2016,75.783,86.831,52665.09,8739806.0,1.631474,0.112024,36227.955632,18625.000,84.0,12.346
2,AUT,2017,76.462,86.632,54188.36,8795073.0,1.560725,0.152499,36984.312014,18974.750,84.0,12.709
3,AUT,2018,77.280,87.348,56956.11,8837707.0,1.552325,0.182909,38190.870562,20416.620,84.0,12.541
4,AUT,2019,77.815,87.949,59716.25,8877637.0,1.557053,0.192745,40065.855276,21946.230,84.0,11.999
...,...,...,...,...,...,...,...,...,...,...,...,...
305,SVK,2013,62.582,89.959,28021.09,5413392.5,0.797937,0.258909,19313.861573,10224.650,36.0,12.634
306,SVN,2016,83.836,96.222,33942.77,2064241.0,0.843163,0.144241,23387.750562,11487.920,55.0,8.045
307,JPN,2017,82.985,NaN,41531.22,126918546.0,0.425807,0.937593,29915.592624,18888.400,992.0,14.515
308,HUN,2015,81.185,89.694,26798.85,9843025.0,0.545663,0.321258,17974.185766,8815.057,69.0,12.438


In [45]:
enrollment_rates_df = pd.read_csv('enrollment_completion_rates.csv')

In [46]:
enrollment_rates_df.head()

,Entity,Code,Year,"Primary completion rate, total (% of relevant age group)","Completion rate, upper secondary education, both sexes (%)","Completion rate, lower secondary education, both sexes (%)","School enrollment, primary (% gross)","School enrollment, secondary (% gross)","School enrollment, tertiary (% gross)"
0,Afghanistan,AFG,1974,16.65700,NaN,NaN,33.10830,10.91069,1.02260
1,Afghanistan,AFG,1977,17.88076,NaN,NaN,36.11149,13.13156,1.40822
2,Afghanistan,AFG,1978,19.65284,NaN,NaN,37.63702,13.82176,1.80833
3,Afghanistan,AFG,1980,26.37324,NaN,NaN,44.13337,16.76427,NaN
4,Afghanistan,AFG,1981,32.70612,NaN,NaN,47.71170,19.32814,NaN


In [47]:
oecd_list = ["AUS","AUT","BEL", "CAN", "CHL", "COL", "CRI",
            "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", 
             "HUN", "ISL", "IRL", "ISR", "ITA", "JPN", "KOR", 
            "LVA", "LTU", "LUX", "MEX", "NLD", "NZL",
            "NOR", "POL", "PRT", "SVK","SVN","ESP","SWE",
            "CHE", "TUR","GBR","USA"]

In [48]:
enrollment_rates_df = enrollment_rates_df.loc[enrollment_rates_df['Code'].isin(oecd_list)]

In [49]:
enrollment_rates_df

,Entity,Code,Year,"Primary completion rate, total (% of relevant age group)","Completion rate, upper secondary education, both sexes (%)","Completion rate, lower secondary education, both sexes (%)","School enrollment, primary (% gross)","School enrollment, secondary (% gross)","School enrollment, tertiary (% gross)"
379,Australia,AUS,2010,NaN,85.02,99.15,105.61943,NaN,NaN
380,Australia,AUS,1971,NaN,NaN,NaN,111.19781,NaN,17.09435
381,Australia,AUS,1972,NaN,NaN,NaN,110.64366,NaN,18.38301
382,Australia,AUS,1973,NaN,NaN,NaN,110.00005,NaN,19.59013
383,Australia,AUS,1974,NaN,NaN,NaN,105.23748,NaN,21.81735
...,...,...,...,...,...,...,...,...,...
8963,United States,USA,2014,NaN,NaN,NaN,99.67338,96.92454,88.62687
8964,United States,USA,2020,NaN,NaN,NaN,100.30579,100.50982,87.56766
8965,United States,USA,1988,NaN,NaN,NaN,NaN,NaN,64.80808
8966,United States,USA,1989,NaN,NaN,NaN,NaN,NaN,67.57282


In [50]:
query = """
        SELECT
            Code AS Country,
            Year,
            "Primary completion rate, total (% of relevant age group)" AS "Primary Completion Rate (%)",
            "Completion rate, upper secondary education, both sexes (%)" AS "Upper Secondary Completion Rate (%)",
            "Completion rate, lower secondary education, both sexes (%)" AS "Lower Secondary Completion Rate (%)",
            "School enrollment, primary (% gross)" AS "Primary Enrollment rate (% gross)",
            "School enrollment, secondary (% gross)" AS "Secondary Enrollment rate (% gross)",
            "School enrollment, tertiary (% gross)" AS "Tertiary Enrollment rate (% gross)"
        FROM enrollment_rates_df
        """

enrollment_rates_df = duckdb.sql(query).df()

In [51]:
enrollment_rates_df

,Country,Year,Primary Completion Rate (%),Upper Secondary Completion Rate (%),Lower Secondary Completion Rate (%),Primary Enrollment rate (% gross),Secondary Enrollment rate (% gross),Tertiary Enrollment rate (% gross)
0,AUS,2010,NaN,85.02,99.15,105.61943,NaN,NaN
1,AUS,1971,NaN,NaN,NaN,111.19781,NaN,17.09435
2,AUS,1972,NaN,NaN,NaN,110.64366,NaN,18.38301
3,AUS,1973,NaN,NaN,NaN,110.00005,NaN,19.59013
4,AUS,1974,NaN,NaN,NaN,105.23748,NaN,21.81735
...,...,...,...,...,...,...,...,...
1774,USA,2014,NaN,NaN,NaN,99.67338,96.92454,88.62687
1775,USA,2020,NaN,NaN,NaN,100.30579,100.50982,87.56766
1776,USA,1988,NaN,NaN,NaN,NaN,NaN,64.80808
1777,USA,1989,NaN,NaN,NaN,NaN,NaN,67.57282
